In [4]:
import pandas as pd
import numpy as np
from io import StringIO 

In [6]:
file_path = "anonymized-curriculum-access.txt"  # Replace with the actual path to your text file
with open(file_path, "r") as file:
    data = file.read()
# Preprocess lines with two consecutive spaces
data = data.replace("  ", " None ")
# Create a DataFrame from the preprocessed data
df = pd.read_csv(StringIO(data), sep=" ", header=None,
                 names=["date", "time", "lesson", "value1", "value2", "ip_address"])

In [7]:
df.head()

,date,time,lesson,value1,value2,ip_address
0,2018-01-26,09:55:03,/,1,8,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61
